# Homework 1 - MapReduce (10 pts)

### DUE: 06/26/2018 before class at 2:00pm

In this homework, we will practice MapReduce. In short, you are asked to redo Task 2 of **BDM_HW1_HOF.ipynb** using MapReduce instead of Python's Higher Order Functions.

You are required to turn in this notebook with all the parts filled in between <>. Your notebook must be named BDM\_HW1\_MR_NetID.ipynb

We will be using the provided the Graduation Outcomes data set for cohorts from 2001 through 2006 (Classes of 2005 through 2010) from the NYC Department of Education (the data handle on NYC Open Data is **avir-tzek**). The data is available on NYU Classes as **nyc_grads.csv**.

#### The first ten records of the data are shown below. Please note, the data are sorted by "Demographics", then "Borough", then "Cohort".

In [1]:
DO NOT RUN THIS CELL

,Demographic,Borough,Cohort,Total Cohort,Grads,Regents,Advanced Regents,Local,Dropped Out
0,Borough Total,Bronx,2001,11453,4913,2644,998,2271,2438
1,Borough Total,Bronx,2002,12032,5328,3118,992,2217,2140
2,Borough Total,Bronx,2003,13632,6389,3861,1255,2528,2472
3,Borough Total,Bronx,2004,14364,7448,4625,1395,2823,2303
4,Borough Total,Bronx,2005,15175,8229,5618,1544,2611,2147
5,Borough Total,Bronx,2006,15579,8524,6312,1558,2212,2402
6,Borough Total,Brooklyn,2001,19961,9758,6177,2829,3591,3547
7,Borough Total,Brooklyn,2002,20808,10337,7050,2865,3298,3369
8,Borough Total,Brooklyn,2003,21334,11064,7711,3239,3353,3198
9,Borough Total,Brooklyn,2004,22353,12303,8872,3741,3431,2973


In [3]:
import csv
import numpy as np
import mapreduce as mr

## Task 1 (10 points)

In this data set, we would like to inspect whether there is a correlation between the dropped out percentage and the percentage of students graduated with advanced regents for schools in NYC. Thus, we must compute the ratio of **Dropped Out** and **Advanced Regents** for each borough, and for each year by dividing them by the **Total Cohort** value. Thus, the only useful records for us are those with the **Demographic** value equals to **'Borough Total'**, aka. you must filter the data to this type of demographic first.

Please write a MapReduce job to compute the average ratio of dropped out and advanced regents per borough (i.e. averaging the numbers from 2001 to 2006 for each borough). Your input should be taken directly from **reader**, the CSV data stream. Though you are encouraged to write your helper functions (instead of lambdas) to make your expression concise, global variables are absolutely not allowed. No exception. The output must be exactly as below.

In [5]:
def mapper1(row):
    if row['Demographic'] == 'Borough Total':
        advanced = (float(row['Advanced Regents']) / float(row['Total Cohort']))
        dropped = (float(row['Dropped Out'] )/ float(row['Total Cohort']))
        yield (row['Borough'], (advanced, dropped))
    
    
def reducer1(k2v2):
    borough, ratios = k2v2
    (advanced, dropped) = zip(*ratios)
    return(borough, np.mean(advanced), np.mean(dropped))


with open('nyc_grads.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output = list(mr.run(reader, mapper1, reducer1))

output

[('Bronx', 0.093419808251337486, 0.17134384308218617),
 ('Brooklyn', 0.15470337770864048, 0.14351662251104022),
 ('Manhattan', 0.16519452307558916, 0.1223416853729485),
 ('Queens', 0.1769903541049419, 0.13903528573260707),
 ('Staten Island', 0.23144827521877342, 0.093750600314718144)]